# Load Mall Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd "/content/drive/MyDrive/mall_dataset"

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from scipy.io import loadmat
from matplotlib.colors import LogNorm
import matplotlib.cm as cm
import time
import skimage.measure
import random
import os
from PIL import Image,ImageFilter,ImageDraw,ImageStat
import numpy as np
import h5py
import cv2
import torch
from torch.utils.data import Dataset
import torchvision.transforms.functional as F
import shutil
import torch.nn as nn
from torchvision import models
import scipy.io as io
import PIL.Image as Image
import glob
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter
import scipy
import json
from matplotlib import cm as CM

%matplotlib inline


In [ ]:
#Print some images from the dataset

path_images = './frames'

if not os.path.exists(path_images):
    print(f"The folder {path_images} doesn't exist.")
else:
    imgs = os.listdir(path_images)
    imgs = sorted([file for file in imgs if file.endswith('.jpg')])

    if not imgs:
        print("No image .jpg found in the folder.")
    else:
        selected_imgs = imgs[:5]
        fig, axes = plt.subplots(1, 5, figsize=(20, 5))

        for ax, image in zip(axes, selected_imgs):
            try:
                path = os.path.join(path_images, image)
                img = Image.open(path)
                ax.imshow(img)
                ax.set_title(image)
                ax.axis('off')

            except Exception as e:
                print(f"Error in the image opening {image}: {e}")

        plt.tight_layout()
        plt.show()


In [ ]:
mall_gt = io.loadmat(os.path.join("./", 'mall_gt.mat'))
mall_head_pos = mall_gt['frame'][0]

In [ ]:
#Changing the first index, we have the coordinates of the heads in the corresponding image
mall_head_pos[0][0][0][0][:,0]

## Image of Mall Dataset Examples

In [ ]:
import os
import scipy.io
from PIL import Image
import matplotlib.pyplot as plt

def visualize_images_with_annotations(image_paths, annotations, num_images):
    plt.figure(figsize=(num_images * 6, 6))

    for i in range(num_images):
        img_path = image_paths[i]
        annotation = annotations[i]

        img = Image.open(img_path)

        plt.subplot(1, num_images, i + 1)
        plt.imshow(img)
        plt.title(f"Image {i + 1}")

        for x, y in annotation:
            plt.scatter(round(x), round(y), s=50, marker="x", color="r")

        plt.axis('off')

    plt.tight_layout()
    plt.show()

path_images = './frames'
num_images_to_show = 5


imgs = os.listdir(path_images)


imgs = sorted([file for file in imgs if file.endswith('.jpg')])

selected_imgs = imgs[:num_images_to_show]

image_paths = [os.path.join(path_images, img) for img in selected_imgs]

annotations = [mall_head_pos[i][0][0][0] for i in range(num_images_to_show)]

visualize_images_with_annotations(image_paths, annotations, 5)


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

path = "/content/drive/MyDrive/mall_dataset/frames"

for name in sorted(os.listdir(path)):

    if name.startswith('.ipynb_checkpoints'):
        continue

    print(name)

    img_path = os.path.join(path, name)
    print(f"Image loading: {img_path}")


    img = cv2.imread(img_path)

    if img is None:
        print(f"Error in image loading : {img_path}")
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.imshow(img)
    plt.title(name)
    plt.axis('off')
    plt.show()

    break

# Density Map Function

In [ ]:
heads= mall_head_pos[0]

def density_map(image, heads):
    output = np.zeros(image.shape[:-1])
    x = heads[0][0][0][:,0]
    y = heads[0][0][0][:,1]

    for x_choord, y_choord in zip(x, y):
        density_i = np.zeros(image.shape[:-1])
        try:
            density_i[round(y_choord),round(x_choord)] = 1
        except:
            pass
        nearest_distance = np.min(np.linalg.norm(x_choord - y_choord))
        sigma = 0.3 * nearest_distance
        kernel_size = 3

        density_i = cv2.GaussianBlur(density_i, (kernel_size,kernel_size), sigma)
        output += density_i
    return cv2.GaussianBlur(output, (51,51), sigma)

start = time.time()
ds_map = density_map(img, heads)
print(f"Time: {time.time() - start}")

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.axis("off")
plt.imshow(img)
plt.subplot(1, 2, 2)
plt.axis("off")
_=plt.imshow(ds_map , cmap = cm.jet)

## Downsampled version

In [ ]:
def density_map_1_over_8(image, heads):
    image = skimage.measure.block_reduce(image, (2,2,1), np.max)
    image = skimage.measure.block_reduce(image, (2,2,1), np.max)
    image = skimage.measure.block_reduce(image, (2,2,1), np.max)

    output = np.zeros(image.shape[:-1])
    x = heads[0][0][0][:,0]/8
    y = heads[0][0][0][:,1]/8

    for x_choord, y_choord in zip(x, y):
        density_i = np.zeros(image.shape[:-1])
        try:
            density_i[round(y_choord),round(x_choord)] = 1
        except:
            pass
        nearest_distance = np.min(np.linalg.norm(x_choord - y_choord))
        sigma = 0.3 * nearest_distance # beta = 0.3 suggested by MCNN paper
        kernel_size = 3

        density_i = cv2.GaussianBlur(density_i, (kernel_size,kernel_size), sigma)
        output += density_i
    return cv2.GaussianBlur(output, (5,5), sigma)

start = time.time()
ds_map = density_map_1_over_8(img, heads)
print(f"Time: {time.time() - start}")

# Conversion

In [ ]:
import os
import cv2
from scipy.io import loadmat
import numpy as np


path_images = "/content/drive/MyDrive/mall_dataset/frames/"
path_save = "/content/drive/MyDrive/mall_dataset/density_maps/"




for i, img_name in enumerate(sorted(os.listdir(path_images))):
    if img_name.startswith('.ipynb_checkpoints') or not img_name.endswith('.jpg'):
        continue

    img_path = os.path.join(path_images, img_name)
    print(f"Image loading: {img_path}")

    img = cv2.imread(img_path)
    if img is None:
        print(f"Error in image loading: {img_path}")
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    heads = mall_head_pos[i-1]

    ds_map_1o8 = density_map_1_over_8(img, heads)

    save_path = os.path.join(path_save, img_name[:-4] + '.npy')
    np.save(save_path, ds_map_1o8)

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title(f"Original Image: {img_name}")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(ds_map_1o8, cmap=cm.jet)
    plt.title(f"Density Map: {img_name[:-4]}")
    plt.axis('off')

    plt.show()

    if i % 10 == 0:
        print(f"{i + 1} density maps generated")

print(f"A total of {i + 1} Density Maps generated.")
